## Logging Configuration

In [ ]:
from bigstream.configure_logging import configure_logging

configure_logging(None, True)

## Inputs
---

In [ ]:
# reading data and working with arrays
import os
import zarr, nrrd
import numpy as np
import os

# data paths
base_path = '/Users/goinac/Work/HHMI/bioimagetools/nextflow-modules'
p = f'{base_path}/mylocal/bigstream-testdata3'
fix_path = f'{p}/fix.n5'
ref_ch = 'c0'
fix_s0_subpath = f'{ref_ch}/s0'
mov_path = f'{p}/mov.n5'
mov_s0_subpath = f'{ref_ch}/s0'
exp_factor = 1

out_p = f'{base_path}/mylocal/results/bigstream-python'
os.makedirs(out_p, exist_ok=True)

# load fix data and spacing
fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
fix_meta = fix_zarr[fix_s0_subpath].attrs.asdict()
print(f'Fix attrs: {fix_meta}')

# load mov data and spacing
mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')
mov_meta = mov_zarr[mov_s0_subpath].attrs.asdict()
print(f'Mov attrs: {mov_meta}')


In [ ]:

fix_spacing_s0 = np.array(fix_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
fix_spacing_s1 = fix_spacing_s0 * [2, 2, 2]
fix_spacing_s2 = fix_spacing_s0 * [4, 4, 4]
fix_spacing_s3 = fix_spacing_s0 * [8, 8, 8]
fix_spacing_s4 = fix_spacing_s0 * [16, 16, 16]
fix_spacing_s5 = fix_spacing_s0 * [32, 32, 32]
print(f'fix_spacing_s0: {fix_spacing_s0}')
print(f'fix_spacing_s1: {fix_spacing_s1}')
print(f'fix_spacing_s2: {fix_spacing_s2}')
print(f'fix_spacing_s3: {fix_spacing_s3}')
print(f'fix_spacing_s4: {fix_spacing_s4}')
print(f'fix_spacing_s5: {fix_spacing_s5}')

mov_spacing_s0 = np.array(mov_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
mov_spacing_s1 = mov_spacing_s0 * [2, 2, 2]
mov_spacing_s2 = mov_spacing_s0 * [4, 4, 4]
mov_spacing_s3 = mov_spacing_s0 * [8, 8, 8]
mov_spacing_s4 = mov_spacing_s0 * [16, 16, 16]
mov_spacing_s5 = mov_spacing_s0 * [32, 32, 32]

print(f'mov_spacing_s0: {mov_spacing_s0}')
print(f'mov_spacing_s1: {mov_spacing_s1}')
print(f'mov_spacing_s2: {mov_spacing_s2}')
print(f'mov_spacing_s3: {mov_spacing_s3}')
print(f'mov_spacing_s4: {mov_spacing_s4}')
print(f'mov_spacing_s5: {mov_spacing_s5}')

global_scale = 's0'
fix_affine_spacing = fix_spacing_s0
mov_affine_spacing = mov_spacing_s0

local_scale = 's0'
fix_deform_spacing = fix_spacing_s0
mov_deform_spacing = mov_spacing_s0

## Alignment
---

### Load affine

In [ ]:
# load precomputed results
affine = np.loadtxt(out_p+'/affine.mat')
affine

### Prepare cluster args

In [ ]:
cluster_kwargs={
    'config':{
        'distributed.worker.memory.target':0.9,
        'distributed.worker.memory.spill':0.9,
        'distributed.worker.memory.pause':0.9,
        'distributed.comm.retry.count':10,
        'distributed.comm.timeouts.connect':'600s',
        'distributed.scheduler.worker-saturation': 1,
        'distributed.scheduler.unknown-task-duration':'60m',
    },
}

### Calculate deform transformation

In [ ]:
%%time
from bigstream.piecewise_align import distributed_piecewise_alignment_pipeline

# FASTER

# get full resolution data
fix = fix_zarr[f'/{ref_ch}/{local_scale}']
mov = mov_zarr[f'/{ref_ch}/{local_scale}']

# define alignment steps
affine_kwargs = {
    'smooth_sigmas':[0.25],
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':25,
    },
    "alignment_spacing": 1.0,
}

deform_kwargs = {
    'smooth_sigmas':[0.25],
    'control_point_spacing':50.0,
    'control_point_levels':[1],
    'optimizer_args':{
        'learningRate':2.5,
        'minStep':0.,
        'numberOfIterations':25,
    },
    "alignment_spacing": 2.0,
}

steps = [ ('affine', affine_kwargs,), ('deform', deform_kwargs,), ]

# deform
deform = distributed_piecewise_alignment_pipeline(
    fix, mov,
    fix_deform_spacing, mov_deform_spacing,
    steps=steps,
    blocksize=[128, 128, 128],
    static_transform_list=[affine,],
    write_path=out_p+'/deform.zarr',
    cluster_kwargs=cluster_kwargs,
)


In [ ]:
# read already saved result
deform = zarr.open(out_p+'/deform.zarr', mode='r')

In [ ]:
print('Deform[0:10,0,0]',deform[0:10,0,0])

In [ ]:
from bigstream.piecewise_transform import distributed_apply_transform

blocksize = (64,)*3
channels = [ref_ch]

fix = fix_zarr[f'/{ref_ch}/{local_scale}']
for channel in channels:
    mov = mov_zarr[f'/{channel}/{local_scale}']
    aligned = distributed_apply_transform(
        fix, mov,
        fix_deform_spacing, mov_deform_spacing,
        transform_list=[affine, deform,],
        transform_spacing=((1,1,1), fix_deform_spacing),
        blocksize=blocksize,
        cluster_kwargs=cluster_kwargs
    )
    print(f'Write aligned result to:{out_p}/r2{channel}{local_scale}_aligned_to_r1.nrrd')
    nrrd.write(out_p+f'/r2{channel}{local_scale}_aligned_to_r1.nrrd', aligned.transpose(2,1,0), compression_level=2)
